In [2]:
import pandas as pd
#Đọc file
khach_hang = pd.read_csv("customer_info.csv")
hoa_don = pd.read_csv("invoice_header.csv")
chi_tiet = pd.read_csv("invoice_detail.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'customer_info.csv'

# **1**.Kết hợp customer_info với invoice_header theo customer_id.

In [ ]:
du_lieu = pd.merge(hoa_don, khach_hang, on="customer_id", how="left")

# **2**.Merge thêm invoice_detail để có bảng dữ liệu hoàn chỉnh.

In [ ]:
du_lieu = pd.merge(du_lieu, chi_tiet, on="invoice_id", how="left")

# **3**. Phát hiện các trường hợp Dữ liệu Không Khớp

In [ ]:
# Chuẩn hóa tên khách hàng và vùng (Không tạo ra NaN mới)
du_lieu["full_name"] = du_lieu["full_name"].astype(str).str.strip().str.title()
du_lieu["region"] = du_lieu["region"].astype(str).str.strip().str.title()

# 1. Làm sạch và chuyển đổi cột số lượng (quantity)
# Các giá trị như ' 2 ', '3a' sẽ chuyển thành NaN
du_lieu["quantity_goc"] = du_lieu["quantity"].copy()
du_lieu["quantity"] = pd.to_numeric(du_lieu["quantity"].astype(str).str.strip(), errors="coerce")

    # 2. Làm sạch và chuyển đổi cột đơn giá (unit_price)
    # Các giá trị có 'vnd' sẽ được làm sạch
du_lieu["unit_price_goc"] = du_lieu["unit_price"].copy()
du_lieu["unit_price"] = du_lieu["unit_price"].astype(str).str.replace("vnd", "").str.strip()
du_lieu["unit_price"] = pd.to_numeric(du_lieu["unit_price"], errors="coerce")

    # Kiểm tra dữ liệu thiếu
print("--- Phát hiện các trường hợp Dữ liệu Không Khớp ---")
tong_nan_truoc_loc = du_lieu.isna().sum()

    # Các cột có dữ liệu thiếu (chỉ liệt kê các cột liên quan đến Merge và Clean)
cac_cot_kiem_tra = ['dob', 'region', 'quantity', 'unit_price']

print("### Tổng hợp giá trị thiếu (NaN) trước khi loại bỏ:")
print(tong_nan_truoc_loc[cac_cot_kiem_tra].to_string())
print("-" * 50)


# **Nguyên** **Nhân**

In [ ]:
# 1. NaN trong quantity (Số lượng):
nan_quantity = du_lieu[du_lieu['quantity'].isna()]
print(f"1. Số dòng có lỗi Số Lượng (quantity là NaN): {nan_quantity.shape[0]} dòng.")
print("   -> Nguyên nhân: Các giá trị số lượng có ký tự chữ ('3a') hoặc khoảng trắng không chuẩn (' 2 ') đã bị chuyển thành NaN do `pd.to_numeric(errors='coerce')` khi làm sạch. Ví dụ: '3a' , ' 2 '")
print("-" * 50)

#2. NaN trong dob và region (Thông tin Khách hàng):
nan_khach_hang = du_lieu[du_lieu['region'].isna()]
print(f"2. Số dòng thiếu thông tin Vùng (region là NaN): {nan_khach_hang.shape[0]} dòng.")
print("   -> Nguyên nhân: Dữ liệu bị thiếu trực tiếp trong file gốc `customer_info.csv`  (Ví dụ: khách hàng C110, C117, C123 ).")
print("-" * 50)

# 3. NaN trong unit_price, product_name, date, invoice_id:
# (Đoạn code bạn cung cấp dùng LEFT JOIN và dữ liệu thực tế cho thấy các cột này không có NaN)
nan_detail = du_lieu[du_lieu['product_name'].isna()]
print(f"3. Số dòng thiếu Chi Tiết Hóa Đơn (product_name là NaN): {nan_detail.shape[0]} dòng.")
print("   -> Nguyên nhân: Số lượng bằng 0, cho thấy tất cả các hóa đơn (invoice_id) trong header đều có chi tiết (item) trong file detail.")
print("-" * 50)


#Loại bỏ dòng lỗi hoặc thiếu dữ liệu quan trọng ---
# Chỉ loại bỏ các dòng có lỗi số lượng/giá để giữ lại các thông tin thiếu khác (region/dob)
# Loại bỏ 10 dòng có quantity lỗi
du_lieu_sach = du_lieu.dropna(subset=["quantity", "unit_price"])

print(f"✅ Số lượng dòng bị loại bỏ (quantity/unit_price là NaN): {du_lieu.shape[0] - du_lieu_sach.shape[0]} dòng.")
print(f"Tổng số dòng còn lại sau khi làm sạch: {du_lieu_sach.shape[0]}.")

# Tính tổng tiền
du_lieu_sach["tong_tien"] = du_lieu_sach["quantity"] * du_lieu_sach["unit_price"]

# Xử lý ngày tháng cho Pivot
du_lieu_sach['date'] = pd.to_datetime(du_lieu_sach['date'], errors='coerce', dayfirst=True)

# Chọn và đổi tên cột cho Pivot
du_lieu_pivot = du_lieu_sach[[
        'invoice_id', 'customer_id', 'full_name', 'region', 'product_name',
        'date', 'quantity', 'unit_price', 'tong_tien'
    ]].copy()

du_lieu_pivot.columns = [
    'Ma_Hoa_Don', 'Ma_Khach_Hang', 'Ten_Khach_Hang', 'Vung', 'Ten_San_Pham',
    'Ngay_Lap_Hoa_Don', 'So_Luong', 'Don_Gia', 'Tong_Tien'
]
du_lieu_pivot


**4. Xuất file dữ liệu hoàn chỉnh dùng cho nhiệm vụ Pivot.**

In [ ]:
# Lưu file kết quả
du_lieu_pivot.to_csv("cleaned_invoice_data.csv", index=False)
print("✅ Dữ liệu đã được làm sạch và lưu vào file 'cleaned_invoice_data.csv'.")
